# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに向けて、未知のエンティティを特定するための言語モデルを開発し、改善することを目的としています。具体的には、Qwen 2 7b - Instructモデルを使用し、質問者、回答者、推測者の役割を持つエージェントを設計しています。

### 問題の取り組み
このノートブックは、言語モデルにおいて複雑な2対2のチーム戦を効果的にプレイするために、効率的な情報収集と推理が求められる「20の質問」ゲームの戦略を検討しています。特に、ルールに基づいて正しい推測を行い、エージェント同士が協力しながら勝利を目指すという挑戦が提示されています。

### 手法とライブラリ
- **使用モデル**: 主にQwen 2 7b - Instructを使用しており、Hugging Faceの他のモデルへの切り替えも可能です。
- **戦略**: 
  - 質問者エージェントが初期質問を行い、その後はゲーム履歴に基づいて質問を生成します。
  - 回答者エージェントは、過去の質問と回答を考慮しつつ、最も多数の意見を基にした回答を生成します。
  - 推測者エージェントはゲーム履歴を参照して推測を行います。
- **ライブラリ**: `transformers`, `torch`, `huggingface_hub`などが使用され、これによりモデルのトークナイザーや重みをロードしています。また、量子化を行うために`BitsAndBytes`ライブラリも利用されています。

### コードの構成
- **依存関係のインストール**: `accelerate`と`bitsandbytes`をインストールしています。
- **モデルのダウンロード**: `huggingface_hub`からモデルをダウンロードし、適切な形式で保存します。
- **メインスクリプトの作成**: Kaggle環境内で動作するエージェントのインターフェースを作成し、契約されたメモリ制限内でモデルを使用するための設定が含まれています。
- **エージェントの初期化と実行**: 観察オブジェクトを用いて、ゲームの各ターンにおける質問、回答、推測を処理します。また、モデルの生成テキストとそのパラメータ調整も行っています。

### 改善点
- より堅牢なモデルの試験、異なるルールや戦略の追加、ゲームの状態追跡機能の実装などが提案されています。これにより、モデルの性能向上が期待できます。

このノートブックは、言語モデルが「20の質問」という複雑なタスクで効果的に機能するための土台を築くことを目指し、モデルの改善と戦術的な応答生成に焦点を当てています。

---


# 用語概説 
以下は、Jupyter Notebook内で使われている専門用語の解説です。初心者の方がつまずきやすいが、ある程度の知識を有する方に向けた解説です。

1. **オーバーフィットモデル**:
   - モデルがトレーニングデータに対して過剰に適合し、新しいデータや汎用性に苦しむ状況を指します。特に公のリーダーボードでのパフォーマンスは良好でも、普遍性がない場合に問題が発生します。

2. **量子化 (Quantization)**:
   - モデルのパラメータ（例:重み）を低いビット数で表現することにより、メモリ使用量と計算の効率を向上させる手法です。例として、4ビットや8ビットの量子化があります。

3. **bitsandbytes**:
   - 大規模なトランスフォーマーモデルを効率的にメモリにロードするためのライブラリです。特に、量子化技術を利用して大きなモデルをサポートします。

4. **T4 GPU**:
   - 特にディープラーニング向けに設計されたNVIDIAのGPUの一種です。推論やデータ処理を高速化するために広く使用されます。

5. **MAX_NEW_TOKENS**:
   - テキスト生成において新たに生成されるトークンの最大数を指定するパラメータです。この数が多いほど長い出力が期待されます。

6. **温度 (Temperature)**:
   - テキスト生成におけるランダム性を制御するパラメータです。低い値は決定的な出力を生成し、高い値は創造的で多様な出力を促します。

7. **デバッグ (Debug)**:
   - プログラムの不具合や動作を確認するプロセスです。このNotebook内では、コード実行の各段階で情報を表示し、問題点を明らかにするために用いられています。

8. **自己対戦 (Self-play)**:
   - 同じエージェントが自分自身と対戦することで、性能向上や最適化を行う手法です。このコンペティションでは、エージェントが自己対戦を通じて評価されます。

9. **ハードコーディング (Hardcoding)**:
   - プログラム内部に固定値やコードを直接記述することです。柔軟性には欠けるが、特定の値を使う場合に簡易な実装が可能になります。

10. **スナップショット (Snapshot)**:
    - モデルやその重みの状態を特定の時点で「スナップショット」として保存し、後に復元または使用できる形です。モデルのデプロイや評価時に便利です。

11. **エージェント (Agent)**:
    - 指定された役割（質問者、回答者、推測者など）を持つプログラム部分を指し、この場合は游戏の状況に応じて行動します。

これらの用語は初心者にとって特に重要ですが、実務での経験が浅いために直感的に理解しにくい部分もあります。注意深く取り扱うことが求められます。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Getting Started
This notebook is submission ready and contains verbose instructions for improvement strategies and ideas to consider. The model is set to Qwen 2 7b - Instruct but can be modified with other models from huggingface.

If you make no changes to this notebook, you can run all your cells or commit your notebook and submit it to the competition. 

## What kind of score will this notebook get? 

This is a difficult question to answer. The keyword search space is large. Unknown entity identification is difficult for language models, and this competition has the increased complexity of playing in cooperation with another agent in the same game vs. another agent team. This 2x2 set up create a challenging context that leads to varied game outcomes. Many discussion posts have been written about this. In addition, there are overfit models that rely on a keyword list to generate guesses. This will earn points in the public leaderboard but fail when the agents are given a secret keyword list. This can make the public leaderboard a poor benchmark for model performance. I recommend that you judge your model performance on self vs. self evaluation. 
1. Do the questions seem logical? 
2. Do the guesses follow the questions? 
3. Is your agent answering questions appropriately? 
4. If you make changes, is your model better now than in the past? 

The answers to these questions won't give your a score or a medal but are at the heart of the true problem we are trying to solve: 

**engineer a small language model to play a unknown entity identification game with another language model, and be good at it.** 

## Submission Script

The script below creates a .py file that will run within the Kaggle environment for the game 20 questions. It utilizes an LLM (downloaded below) to generate text during each round. The script includes a class that interacts with the structure and variables given in the game environment. It also includes three functions that prompt the LLM into taking the role of questioner, answerer, and guesser. Lastly, it includes text generation using the initialized model and tokenizer, and the proper agent calls from the game. 

### Strategies

The strategies utilized in this script include:
* 'hard-coded' first questions that helps get the game started and provide a few-shot example to the LLM of how the game should proceed. 
* a questioner and guesser agent the review the game history to create a response.
* an answerer agent that loops 5 times and returns the most frequent response.
* model is loaded into cuda:0 to simulate the competition limitations. 

Not in this script: 
* supplying the keywords to the LLM to improve it's performance on the public leaderboard.
* supplying external words list to provide a robust keyword list for guesser to search. 


## Areas for Improvement
* Add a more robust LLM model. Review the huggingface open LLM leaderboard for ~7B or lower models and attempt to load them into the script. There will be issues with loading some models, but it can be fun to tinker and problem solve. 
* Try Phi3, Gemma , or LLaMa 3 and see how it performs compared to Gwen. (Make sure to download the libraries to the sumbission folder!)
* If you add a larger model, make sure to follow installing/importing bitsandbytes and accelerate and load the model with quantization-config set to 4bit. 
* Add game state tracking that defines what category the keyword is likely to be in based on the answers provided. 
* Add game phase tracking to move from broad early game questions to narrow specific questions in late game. 
* Add multi-step prompting rather than straight-to-response text generation.
* Experiment with text generation parameters like temparature. 
* You'll notice that the intruction tuned Qwen model can sometimes struggle with multi-level prompts. Will other models 'chat' better with itself? 
* A fine tuned model. Out-of-the-box models aren't made specifically to be good at 20 questions. Some are better than others. Maybe a dataset of Q&A pairs from a ongoing 20 questions competition would be helpful?????

## FAQ
* Getting an out of memory error? Do a factory reset and run the script again. You probably loaded the model twice somehow! 
* Why are some of the responses from the questioner or guesser not formatted right? You need to find a creative way to parse the response. This can be done throught prompting or hard-coding a parser that extracted the valid response from the generated text. 

# Let's get to the code

### First - Install Dependencies
We will install some dependencies for the script AND install them into a temporary directory that will be uploaded with the submission. This is essential to ensure that the script runs in the Kaggle environment.
If you need to install other packages, make sure to add them to both cells below. 

</div>
<div class="column-right">

# 日本語訳

# はじめに
このノートブックは提出の準備が整っており、改善戦略や検討すべきアイデアに関する詳しい指示が含まれています。モデルはQwen 2 7b - Instructに設定されていますが、huggingfaceの他のモデルに変更することも可能です。

このノートブックに変更を加えなければ、すべてのセルを実行するかノートブックをコミットしてコンペティションに提出できます。

## このノートブックはどのようなスコアになりますか？

これは難しい質問です。キーワードの検索空間は広大で、未知のエンティティの特定は言語モデルにとって難しいです。また、このコンペティションでは、異なるエージェントチームの中で協力してプレイするという複雑さが加わります。この2対2の設定は、さまざまなゲームの結果を引き起こす挑戦的な文脈を作り出します。この件について多くの議論がされてきました。その上、キーワードリストに依存して推測を生成するオーバーフィットモデルもあります。これらは、公のリーダーボードではポイントを獲得しますが、エージェントに秘密のキーワードリストが与えられると失敗します。これにより、公のリーダーボードはモデルの性能の良いベンチマークにならない可能性があります。自己評価によってモデルの性能を判断することをお勧めします。
1. 質問は論理的に思われますか？
2. 推測は質問に従っていますか？
3. あなたのエージェントは適切に質問に答えていますか？
4. 変更を加えた場合、以前よりもモデルは良くなりましたか？

これらの質問への回答はスコアやメダルを与えるものではありませんが、私たちが解決しようとしている真の問題の核心にあります：

**小さな言語モデルを開発し、未知のエンティティ特定ゲームをプレイし、そのスキルを向上させることです。**

## 提出スクリプト

以下のスクリプトは、20の質問ゲーム用にKaggle環境内で実行される.pyファイルを作成します。このスクリプトは、各ラウンドでテキストを生成するために（以下からダウンロードされた）LLMを利用します。スクリプトには、ゲーム環境内で提供される構造と変数と相互作用するクラスが含まれています。また、質問者、回答者、推測者の役割を取るためにLLMを促す3つの関数も含まれています。最後に、初期化されたモデルとトークナイザーを使用したテキスト生成、およびゲームからの適切なエージェントコールが含まれています。

### 戦略

このスクリプトで利用される戦略には以下が含まれます：
* ゲームスタートを助け、LLMにゲームの進め方を示すための「ハードコーディングされた」最初の質問。
* ゲーム履歴をレビューして応答を作成する質問者エージェントと推測者エージェント。
* 5回ループして最も頻繁な応答を返す回答者エージェント。
* コンペティションの制限をシミュレーションするためにcuda:0にモデルをロード。

スクリプトには含まれていない：
* 公のリーダーボードでのパフォーマンスを向上させるためにLLMにキーワードを供給すること。
* 推測者が検索するための堅牢なキーワードリストを提供する外部単語リストの供給。 


## 改善点
* より堅牢なLLMモデルを追加。huggingfaceのオープンLLMリーダーボードで約7Bまたはそれ以下のモデルをレビューし、それらをスクリプトにロードしてみてください。特定のモデルのロードに問題があるでしょうが、試行錯誤して問題解決を楽しむことができます。
* Phi3、Gemma、またはLLaMa 3を試して、Gwenと比較してどうなるか確認してください。（提出フォルダにライブラリをダウンロードしておくことを忘れずに！）
* より大きなモデルを追加する場合、bitsandbytesとaccelerateをインストールおよびインポートし、quantization-configを4ビットに設定してモデルをロードしてください。
* 提供された回答に基づいて、キーワードがどのカテゴリに属する可能性が高いかを定義するゲーム状態追跡を追加してください。
* 初期の広範なゲーム質問から、後半の具体的な質問に移行するためのゲームフェーズ追跡を追加してください。
* 直行テキスト生成の代わりにマルチステッププロンプトを追加してください。
* temperatureなどのテキスト生成パラメータを試してみてください。
* インストラクションチューニングされたQwenモデルは、時折マルチレベルプロンプトに苦労することがあります。他のモデルは自己対話に優れているでしょうか？
* 微調整されたモデル。標準のモデルは20の質問のプレイに特化していません。いくつかは他のモデルよりも優れています。進行中の20の質問コンペティションからのQ&Aペアのデータセットが役立つかもしれませんか？

## よくある質問
* メモリエラーが表示されますか？ファクトリーリセットを行い、スクリプトを再実行してください。おそらくモデルを二重にロードしてしまったのです！
* 質問者または推測者からのいくつかの応答が適切にフォーマットされていないのはなぜですか？応答をパースするための創造的な方法を見つける必要があります。これはプロンプトを使用するか、生成されたテキストから有効な応答を抽出するためにハードコーディングされたパーサを使用することで実現できます。

# コードに進みましょう

### 最初 - 依存関係のインストール
スクリプト用の依存関係をいくつかインストールし、提出時にアップロードされる一時ディレクトリにもインストールします。これは、スクリプトがKaggle環境内で正常に実行されるために必須です。
他のパッケージをインストールする必要がある場合は、下記の両方のセルに追加してください。 


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install accelerate bitsandbytes 
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install accelerate bitsandbytes
```

</div>
</details>

In [ ]:
!pip install accelerate bitsandbytes

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

The cell takes a bit longer. 

</div>
<div class="column-right">

# 日本語訳

このセルは少し時間がかかります。 


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
os.system("pip install -t /tmp/submission/lib accelerate bitsandbytes")
```

</div>
<div class="column-right">

# 日本語訳

```python
import os
os.system("pip install -t /tmp/submission/lib accelerate bitsandbytes")
```

</div>
</details>

In [ ]:
import os
os.system("pip install -t /tmp/submission/lib accelerate bitsandbytes")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Snapshot Download: What does this cell do?

This section downloads and saves the model to a temporary directory that gets zipped into the submission folder and is called by the script after submission. Since the game environment is run in a Kaggle environment, the model weights must be downloaded and submitted. This cell takes a few minutes to load.

Note: this does not load the model or tokenizer. That step happens in the main script. 

Currently, the model is set to Qwen 2 7b Instruct. 

</div>
<div class="column-right">

# 日本語訳

### スナップショットダウンロード：このセルは何をしますか？

このセクションでは、モデルを1つの提出フォルダに圧縮（zip）した後、提出後にスクリプトによって呼び出される一時ディレクトリにダウンロードして保存します。ゲーム環境はKaggle環境で実行されるため、モデルの重みはダウンロードして提出しなければなりません。このセルの読み込みには数分かかります。

注：これはモデルやトークナイザーを読み込むわけではありません。そのステップはメインスクリプトで行われます。

現在、モデルはQwen 2 7b Instructに設定されています。 


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

model_path = Path("/tmp/submission/")
if model_path.exists():
    shutil.rmtree(model_path)
model_path.mkdir(parents=True)

snapshot_download(
    repo_id= "Qwen/Qwen2-7B-Instruct",
    local_dir=model_path
)
```

</div>
<div class="column-right">

# 日本語訳

```python
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

model_path = Path("/tmp/submission/")
if model_path.exists():
    shutil.rmtree(model_path)
model_path.mkdir(parents=True)

snapshot_download(
    repo_id= "Qwen/Qwen2-7B-Instruct",
    local_dir=model_path
)
```

</div>
</details>

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

model_path = Path("/tmp/submission/")
if model_path.exists():
    shutil.rmtree(model_path)
model_path.mkdir(parents=True)

snapshot_download(
    repo_id= "Qwen/Qwen2-7B-Instruct",
    local_dir=model_path
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# The Main Script 

The magic command at the top of the notebook will create a .py file that will run within the Kaggle environment and be submitted to the competition.

- Running this script for evaluation loads the quantized model into GPU memory. Since Gwen 2 7b is large, the 4 bit quantized verison is loaded and takes up 7.4 GB of memory of the 15 GB available on the single T4. 

</div>
<div class="column-right">

# 日本語訳

# メインスクリプト 

ノートブックの最上部のマジックコマンドは、Kaggle環境内で実行され、コンペティションに提出される.pyファイルを作成します。

- このスクリプトを評価のために実行すると、量子化されたモデルがGPUメモリにロードされます。Qwen 2 7bは大きいため、4ビットの量子化バージョンがロードされ、15GBのうち7.4GBのメモリを使用します。 


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%writefile main.py

# Comment out the line above if performing an evaluation session. 

import os

#This checks to see what environment this script is running in. That way you can submit and evaluate in different environments.
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if not os.path.exists(KAGGLE_AGENT_PATH):
    KAGGLE_AGENT_PATH = "/tmp/submission/"

import sys
import itertools
import re
import torch
import typing as t
from pathlib import Path
from pydantic import BaseModel, Field, field_validator
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from IPython.display import display, Markdown

model_initialized = False
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t_dtype = torch.bfloat16
DEBUG = False

#==================QUANTIZATION CONFIG================
# You need to quantize Gwen 2 7b. It is too big to fit on the single T4 GPU used in the competition.
# This is included for reference so that you can see what the config looks like to quantize a larger model.
# We also deploy a conditional statement that checks if the model is already loaded. If it is, we skip loading to prevent OOM errors.
# This is super helpful when modifying the script, checking the evaluation session, and trying something new. 

if 'model' in locals() and 'tokenizer' in locals() and model is not None and tokenizer is not None:
    print("Model and tokenizer already exist. Skipping creation.")
else:
    quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                             bnb_4bit_compute_dtype=t_dtype)
    
    model = AutoModelForCausalLM.from_pretrained(KAGGLE_AGENT_PATH, 
                                                 device_map=device, 
                                                 torch_dtype=t_dtype,
                                                 quantization_config=quantization_config)
    
    tokenizer = AutoTokenizer.from_pretrained(KAGGLE_AGENT_PATH, 
                                              torch_dtype=t_dtype)

        
#==================OBSERVATION CLASS FOR KAGGLE ENVIRONMENT================
# This class takes the obs input from the kaggle environment for use in this script.
# Shout out to Team Rigging public notebook for the observation class.

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_text(self, *, skip_guesses: bool = False, skip_answer: bool = False, skip_question: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""{'**Question:** ' + question if not skip_question else ''} -> {'**Answer:** ' + answer if not skip_answer else ''}
            {'**Previous Guess:** ' + guess if not skip_guesses else ''}
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "none yet."


# =================SYSTEM PROMPT================
# Here is your system prompt that is inserted into the message template during text generation.
# For instruction/chat models, this helps provide context and direction for the model to follow.
# There are many prompt strategies to deploy here that can be explored and researched. 
# You can also have different ssystem prompts for different roles, game states, or game phases (early/mid/late game).

system_prompt = """You are playing the interactive unknown entity identification game, 20 questions.
You are an experienced player of all roles, questioner, answerer, and guesser.
Your objective is to guess the secret keyword in as few turns as possible and win."""

# =================ASKER AGENT================
# The asker agent first asks is the keyword a place then asks if it is located north of the equator. No LLM is called here.
# If the answer to the place quesiton is no, it asks if it is a non-living thing.
# This helps provide a game environment few-shot example to the LLM AND helps bisect the first categories.
# Afterwards (steps > 1), it asks a generated question based on the previous history.
# You could add additional conditional questions before calling the LLM to further bisect the search space.
# You could also add text generation steps to initiate chain-of-thought reasoning, or other techniques.
# Make sure to check your max_new_tokens and temperature parameters.
# As your prompt complexity increases, you will want to create different generate_text functions, with different parameters.
# Like a generate_long with has much higher new token limit and higher temperature.

def ask(observation: Observation) -> str:
    if observation.step == 0:
        return "Is the secret keyword a place?"
    elif observation.step == 1:
        if len(observation.answers) > 0 and observation.answers[0] == "yes":
            return "Is it a city?"
        else:
            return "Is it a living thing?"
    game_history = observation.get_history_text(skip_guesses=True)

    try:
        think = generate_text(f"""
        You are playing as questioner and currently asking the next question. Here's what you know so far:

====== GAME HISTORY =====
{game_history}
=========================

        Based on the history above, explain which direction the questions should go next. 
        """)
        
        parse = generate_text(f"""
        Review the current information below and give your next best question. 
        If you don't have much information, it is best to ask a yes/no question that eliminates categories from the search space.  

        Summary: "{think}""
        
        Just ask a question. Don't provide any more text than the question and a question mark. 
        """
        ).lower()
        if DEBUG:
            display(Markdown(f"### DEBUG: Questioner thinking:"))
            display(Markdown(think))
            display(Markdown(f"### DEBUG: Questioner parsing:"))
            display(Markdown(parse))
        return parse
    except Exception as e:
        print(f"Error while asking question: {e}")
        return 'Is it a place?'
    

# =================ANSWERER AGENT================
# Loop 5 times and take the most frequent response
# This was a technique used in the Team Rigging notebooks. 
# It is slightly more effective than a single response. 

def answer(observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("Keyword wasn't provided to answerer", file=sys.stderr)
        return "yes"
            
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            response = generate_text(f"""
            20 Questions game. Answer yes/no for this keyword: {observation.keyword} 

            Question: "{last_question}"
            
            Rules:
            1. Only consider your keyword: {observation.keyword} that is in the category: {observation.category}.
            2. Answer 'yes' or 'no. Nothing else. 
            Your answer here:
            """
            ).lower()
            if DEBUG:
                display(Markdown(f"### DEBUG: Answerer response:"))
                display(Markdown(f"DEBUG: {response}"))
            
            yes_no = re.findall(r'\b(yes|no)\b', response)
            if yes_no:
                responses.append(yes_no[0])

        if DEBUG:
            display(Markdown(f"DEBUG: All Answerer responses are {responses}"))
        return max(set(responses), key=responses.count)

    except Exception as e:
        print(f"Error in answerer: {e}", file=sys.stderr)
        return "yes"

# ================GUESSER AGENT================
# This agent uses the game history to make a guess.
# In some instances, this is not helpful because the LLM may get stuck in a associated word loop.
# Try different prompt strategies including setting skip guesses to True. 
# Parsing strategies helps make sure thet the game reponse is valid and understandable.


def guess(observation: Observation) -> str:
    game_history = observation.get_history_text(skip_guesses=False)
    
    think = generate_text(f"""
    You are currently the guesser, making an informed guess of the keyword. 

== GAME HISTORY ==
{game_history}
==================

    Suggest 1 category that is still relevant to the secret keyword based on this information and suggest a possible guess.
    Explain your reasoning. 
    """
    )
    
    parse = generate_text(f"""
    Review the response below and give your next best guess. 
    If you don't have much information, make a wild, but specific, guess.
    Don't ask the guess like a question, just give the word or phrase guess. 

    Summary: "{think}"

    Your guess here:
    """
    ).lower()
    if DEBUG:
        print(f"Game history: {game_history}")
        display(Markdown(f"### DEBUG: Guesser thinking:"))
        display(Markdown(think))
        display(Markdown(f"### DEBUG: Guesser parse:"))
        display(Markdown(parse))
    
    return parse

#==============LLM MESSAGE GENERATION===============
# There are few parameters you may want to modify.
# Temperature sets the randomness of the generated text. 
# Lower values are more deterministic, higher - more creative. 
# max_new_tokens sets the length of the generated text.

    
def generate_text(prompt:str) -> str:
    sys_prompt = system_prompt
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(messages, 
                                         tokenize=False, 
                                         add_generation_prompt=True)
    
    inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(inputs.input_ids,
                                   max_new_tokens=350, 
                                   do_sample=True, 
                                   temperature=0.1) #this sets the randomness/creativity of the generated text, lower is more deterministic
    
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

#================AGENT INITIALIZATION FOR KAGGLE ENV================
# It may be fun to change one of the agents to 'input' to play against the model during eval!
# Bonus, this opens up a possibility to create a fine-tuning dataset! How neat!
# Maybe you could alter the eval session to be a fine-tuning dataset building session that saves the output in a dataframe?
# The possibilities are endless but time is limited.  

agent = None

def observe(obs: t.Any) -> str:
    global agent
    observation = Observation(**obs.__dict__)

    try:
        match observation.turnType:
            case "ask":
                agent = ask(observation)
                return agent
            case "answer":
                agent = answer(observation)
                return agent
            case "guess":
                agent = guess(observation)
                return agent

            case _:
                raise ValueError("Unknown turn type")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    return observe(obs)

# =========================ENABLE TORCH BACKEND===========================
# Sometimes you'll get errors when running the script in this notebook. 
# These backend settings ensure that the script runs without errors. Yay!

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
```

</div>
<div class="column-right">

# 日本語訳

```python
%%writefile main.py

# 上記の行は評価セッションを実行する場合はコメントアウトしてください。

import os

#このスクリプトがどの環境で実行されているかを確認します。これにより、異なる環境で提出と評価を行うことができます。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if not os.path.exists(KAGGLE_AGENT_PATH):
    KAGGLE_AGENT_PATH = "/tmp/submission/"

import sys
import itertools
import re
import torch
import typing as t
from pathlib import Path
from pydantic import BaseModel, Field, field_validator
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from IPython.display import display, Markdown

model_initialized = False
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t_dtype = torch.bfloat16
DEBUG = False

#==================量子化設定================
# Gwen 2 7bを量子化する必要があります。それは、コンペティションで使用される単一のT4 GPUに収まらないからです。
# これは、より大きなモデルを量子化するための構成がどのように見えるかを参照するために含まれています。
# また、モデルが既に読み込まれているか確認する条件文も展開しています。もしそれが真であれば、OOMエラーを防ぐために読み込みをスキップします。
# これは、スクリプトの修正、評価セッションの確認、新しい試みを行う際に非常に役立ちます。

if 'model' in locals() and 'tokenizer' in locals() and model is not None and tokenizer is not None:
    print("モデルとトークナイザーはすでに存在します。作成処理をスキップします。")
else:
    quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                             bnb_4bit_compute_dtype=t_dtype)
    
    model = AutoModelForCausalLM.from_pretrained(KAGGLE_AGENT_PATH, 
                                                 device_map=device, 
                                                 torch_dtype=t_dtype,
                                                 quantization_config=quantization_config)
    
    tokenizer = AutoTokenizer.from_pretrained(KAGGLE_AGENT_PATH, 
                                              torch_dtype=t_dtype)

        
#==================KAGGLE環境用の観察クラス================
# このクラスは、Kaggle環境からのobs入力をこのスクリプトで使用できるようにします。
# 観察クラスはTeam Riggingの公開ノートブックに感謝します。

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_text(self, *, skip_guesses: bool = False, skip_answer: bool = False, skip_question: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""{'**質問:** ' + question if not skip_question else ''} -> {'**回答:** ' + answer if not skip_answer else ''}
            {'**以前の推測:** ' + guess if not skip_guesses else ''}
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "まだなし."


# =================システムプロンプト================
# ここにあなたのシステムプロンプトがあり、テキスト生成中のメッセージテンプレートに挿入されます。
# 指示/チャットモデルの場合、モデルに従うべきコンテキストと方向性を提供します。
# ここで展開できるさまざまなプロンプト戦略がいくつかあります。
# ゲームの状態やフェーズ（初期/中期/後期）に応じて異なるシステムプロンプトを持つことも可能です。

system_prompt = """あなたは、インタラクティブな未知のエンティティ特定ゲーム「20の質問」をプレイしています。
あなたは質問者、回答者、推測者の全役割に熟練した熟練者です。
あなたの目標は、できるだけ少ないターンで秘密のキーワードを推測して勝利することです。"""

# =================質問者エージェント================
# 質問者エージェントは最初にキーワードが場所かどうかを尋ね、その後赤道の北側に位置しているかどうかを尋ねます。ここではLLMは呼び出されません。
# キーワードが「場所」でない場合、それは原因が無生物であるかどうかを尋ねます。
# これによりゲーム環境の少数のサンプルをLLMに提供し、最初のカテゴリを二分するのに役立ちます。
# それ以降（step > 1）、前の履歴に基づいて生成された質問を詢います。
# 追加の条件付き質問をLLMを呼び出す前に加え、探索空間をさらに絞り込むことができます。
# LLMを呼び出す前に、チェーンオブシンク推論や他のテクニックを追加するためのテキスト生成ステップを追加することもできます。
# max_new_tokensやtemperatureパラメータをチェックすることを忘れずに。
# プロンプトの複雑さが増すにつれて、異なるパラメータを持つ異なるgenerate_text関数を作成したいでしょう。
# たとえば、generate_longははるかに高い新しいトークン制限と高い温度を持ちます。

def ask(observation: Observation) -> str:
    if observation.step == 0:
        return "秘密のキーワードは場所ですか？"
    elif observation.step == 1:
        if len(observation.answers) > 0 and observation.answers[0] == "yes":
            return "都市ですか？"
        else:
            return "生きているものですか？"
    game_history = observation.get_history_text(skip_guesses=True)

    try:
        think = generate_text(f"""
        あなたは質問者として次の質問をしており、以下のことを知っています：

====== ゲーム履歴 =====
{game_history}
=========================

        上記の履歴に基づき、次にどのように質問を進めるべきかを説明してください。
        """)
        
        parse = generate_text(f"""
        現在の情報を確認し、次のベストな質問を教えてください。
        あまり情報がない場合は、検索空間からカテゴリを排除するためにはい/いいえの質問をするのがベストです。

        概要: "{think}""
        
        質問をただ聞いてください。質問と疑問符以外のテキストを提供しないでください。
        """
        ).lower()
        if DEBUG:
            display(Markdown(f"### デバッグ: 質問者の思考:"))
            display(Markdown(think))
            display(Markdown(f"### デバッグ: 質問者のパース:"))
            display(Markdown(parse))
        return parse
    except Exception as e:
        print(f"質問を尋ねる際のエラー: {e}")
        return '場所ですか？'
    

# =================回答者エージェント================
# 5回ループし、最も頻繁な応答を取ります。
# これはTeam Riggingのノートブックで使用されたテクニックです。
# 単一の応答よりもわずかに効果的です。

def answer(observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("キーワードが回答者に提供されませんでした", file=sys.stderr)
        return "yes"
            
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            response = generate_text(f"""
            20の質問ゲーム。このキーワードに対してはい/いいえで答えてください: {observation.keyword} 

            質問: "{last_question}"
            
            ルール:
            1. あなたのキーワード: {observation.keyword} にのみ注意してください。これはカテゴリに分けられます: {observation.category}.
            2. 'はい'または'いいえ'だけを答えてください。その他の情報は必要ありません。
            ここにあなたの回答を入力してください:
            """
            ).lower()
            if DEBUG:
                display(Markdown(f"### デバッグ: 回答者の応答:"))
                display(Markdown(f"DEBUG: {response}"))
            
            yes_no = re.findall(r'\b(yes|no)\b', response)
            if yes_no:
                responses.append(yes_no[0])

        if DEBUG:
            display(Markdown(f"DEBUG: すべての回答者の応答 {responses}"))
        return max(set(responses), key=responses.count)

    except Exception as e:
        print(f"回答者でエラー: {e}", file=sys.stderr)
        return "yes"

# ================推測者エージェント================
# このエージェントはゲーム履歴を使用して推測を行います。
# 一部の場合、LLMが関連する単語ループにハマるため、これはあまり役に立たないことがあります。
# 異なるプロンプト戦略を試すことや、推測をスキップすることが良いでしょう。
# パース戦略を使用すると、ゲームの応答が有効で理解可能であることを保証します。

def guess(observation: Observation) -> str:
    game_history = observation.get_history_text(skip_guesses=False)
    
    think = generate_text(f"""
    あなたは現在、推測者としてキーワードの情報に基づいた推測を行います。

== ゲーム履歴 ==
{game_history}
==================

    提案された関連する1つのカテゴリと、可能な推測を提案してください。
    あなたの根拠を説明してください。
    """
    )
    
    parse = generate_text(f"""
    以下の応答を確認し、次のベストな推測を教えてください。
    あまり情報がない場合は、野生の具体的な推測をしてください。
    質問のように推測を尋ねず、単語またはフレーズの推測を直接提供してください。

    概要: "{think}"

    あなたの推測はこちら:
    """
    ).lower()
    if DEBUG:
        print(f"ゲーム履歴: {game_history}")
        display(Markdown(f"### デバッグ: 推測者の思考:"))
        display(Markdown(think))
        display(Markdown(f"### デバッグ: 推測者のパース:"))
        display(Markdown(parse))
    
    return parse

#==============LLMメッセージ生成===============
# 変更したいパラメータがいくつかあります。
# temperatureは生成されるテキストのランダム性を設定します。
# 値の低い方がより決定論的で、高い方がより創造的です。
# max_new_tokensは生成されるテキストの長さを設定します。

    
def generate_text(prompt:str) -> str:
    sys_prompt = system_prompt
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(messages, 
                                         tokenize=False, 
                                         add_generation_prompt=True)
    
    inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(inputs.input_ids,
                                   max_new_tokens=350, 
                                   do_sample=True, 
                                   temperature=0.1) #これは生成テキストのランダム性/創造性を設定します。低いほど決定論的です。
    
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

#================KAGGLE環境用エージェントの初期化================
# 評価中にモデルと対戦してエージェントの1つを'input'に変更すると楽しさが増します！
# ボーナスとして、微調整データセットを作成する可能性が開かれます！素晴らしいですね！
# 評価セッションを微調整データセットの作成セッションにすることを検討できます。これにより、出力をデータフレームに保存できます。
# 無限の可能性がありますが、時間に制約があります。  

agent = None

def observe(obs: t.Any) -> str:
    global agent
    observation = Observation(**obs.__dict__)

    try:
        match observation.turnType:
            case "ask":
                agent = ask(observation)
                return agent
            case "answer":
                agent = answer(observation)
                return agent
            case "guess":
                agent = guess(observation)
                return agent

            case _:
                raise ValueError("不明なターンタイプ")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    return observe(obs)

# =========================TORCHバックエンドを有効化===========================
# このノートブックでスクリプトを実行するときにエラーが発生することがあります。
# これらのバックエンド設定は、エラーなしでスクリプトが実行されることを保証します。やったね！

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
```

</div>
</details>

In [ ]:
%%writefile main.py

# 上記の行は評価セッションを実行する場合はコメントアウトしてください。

import os

#このスクリプトがどの環境で実行されているかを確認します。これにより、異なる環境で提出と評価を行うことができます。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if not os.path.exists(KAGGLE_AGENT_PATH):
    KAGGLE_AGENT_PATH = "/tmp/submission/"

import sys
import itertools
import re
import torch
import typing as t
from pathlib import Path
from pydantic import BaseModel, Field, field_validator
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from IPython.display import display, Markdown

model_initialized = False
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t_dtype = torch.bfloat16
DEBUG = False

#==================量子化設定================
# Gwen 2 7bを量子化する必要があります。それは、コンペティションで使用される単一のT4 GPUに収まらないからです。
# これは、より大きなモデルを量子化するための構成がどのように見えるかを参照するために含まれています。
# また、モデルが既に読み込まれているか確認する条件文も展開しています。もしそれが真であれば、OOMエラーを防ぐために読み込みをスキップします。
# これは、スクリプトの修正、評価セッションの確認、新しい試みを行う際に非常に役立ちます。

if 'model' in locals() and 'tokenizer' in locals() and model is not None and tokenizer is not None:
    print("モデルとトークナイザーはすでに存在します。作成処理をスキップします。")
else:
    quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                             bnb_4bit_compute_dtype=t_dtype)
    
    model = AutoModelForCausalLM.from_pretrained(KAGGLE_AGENT_PATH, 
                                                 device_map=device, 
                                                 torch_dtype=t_dtype,
                                                 quantization_config=quantization_config)
    
    tokenizer = AutoTokenizer.from_pretrained(KAGGLE_AGENT_PATH, 
                                              torch_dtype=t_dtype)

        
#==================KAGGLE環境用の観察クラス================
# このクラスは、Kaggle環境からのobs入力をこのスクリプトで使用できるようにします。
# 観察クラスはTeam Riggingの公開ノートブックに感謝します。

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_text(self, *, skip_guesses: bool = False, skip_answer: bool = False, skip_question: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""{'**質問:** ' + question if not skip_question else ''} -> {'**回答:** ' + answer if not skip_answer else ''}
            {'**以前の推測:** ' + guess if not skip_guesses else ''}
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "まだなし."


# =================システムプロンプト================
# ここにあなたのシステムプロンプトがあり、テキスト生成中のメッセージテンプレートに挿入されます。
# 指示/チャットモデルの場合、モデルに従うべきコンテキストと方向性を提供します。
# ここで展開できるさまざまなプロンプト戦略がいくつかあります。
# ゲームの状態やフェーズ（初期/中期/後期）に応じて異なるシステムプロンプトを持つことも可能です。

system_prompt = """あなたは、インタラクティブな未知のエンティティ特定ゲーム「20の質問」をプレイしています。
あなたは質問者、回答者、推測者の全役割に熟練した熟練者です。
あなたの目標は、できるだけ少ないターンで秘密のキーワードを推測して勝利することです。"""

# =================質問者エージェント================
# 質問者エージェントは最初にキーワードが場所かどうかを尋ね、その後赤道の北側に位置しているかどうかを尋ねます。ここではLLMは呼び出されません。
# キーワードが「場所」でない場合、それは原因が無生物であるかどうかを尋ねます。
# これによりゲーム環境の少数のサンプルをLLMに提供し、最初のカテゴリを二分するのに役立ちます。
# それ以降（step > 1）、前の履歴に基づいて生成された質問を詢います。
# 追加の条件付き質問をLLMを呼び出す前に加え、探索空間をさらに絞り込むことができます。
# LLMを呼び出す前に、チェーンオブシンク推論や他のテクニックを追加するためのテキスト生成ステップを追加することもできます。
# max_new_tokensやtemperatureパラメータをチェックすることを忘れずに。
# プロンプトの複雑さが増すにつれて、異なるパラメータを持つ異なるgenerate_text関数を作成したいでしょう。
# たとえば、generate_longははるかに高い新しいトークン制限と高い温度を持ちます。

def ask(observation: Observation) -> str:
    if observation.step == 0:
        return "秘密のキーワードは場所ですか？"
    elif observation.step == 1:
        if len(observation.answers) > 0 and observation.answers[0] == "yes":
            return "都市ですか？"
        else:
            return "生きているものですか？"
    game_history = observation.get_history_text(skip_guesses=True)

    try:
        think = generate_text(f"""
        あなたは質問者として次の質問をしており、以下のことを知っています：

====== ゲーム履歴 =====
{game_history}
=========================

        上記の履歴に基づき、次にどのように質問を進めるべきかを説明してください。
        """)
        
        parse = generate_text(f"""
        現在の情報を確認し、次のベストな質問を教えてください。
        あまり情報がない場合は、検索空間からカテゴリを排除するためにはい/いいえの質問をするのがベストです。

        概要: "{think}""
        
        質問をただ聞いてください。質問と疑問符以外のテキストを提供しないでください。
        """
        ).lower()
        if DEBUG:
            display(Markdown(f"### デバッグ: 質問者の思考:"))
            display(Markdown(think))
            display(Markdown(f"### デバッグ: 質問者のパース:"))
            display(Markdown(parse))
        return parse
    except Exception as e:
        print(f"質問を尋ねる際のエラー: {e}")
        return '場所ですか？'
    

# =================回答者エージェント================
# 5回ループし、最も頻繁な応答を取ります。
# これはTeam Riggingのノートブックで使用されたテクニックです。
# 単一の応答よりもわずかに効果的です。

def answer(observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("キーワードが回答者に提供されませんでした", file=sys.stderr)
        return "yes"
            
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            response = generate_text(f"""
            20の質問ゲーム。このキーワードに対してはい/いいえで答えてください: {observation.keyword} 

            質問: "{last_question}"
            
            ルール:
            1. あなたのキーワード: {observation.keyword} にのみ注意してください。これはカテゴリに分けられます: {observation.category}.
            2. 'はい'または'いいえ'だけを答えてください。その他の情報は必要ありません。
            ここにあなたの回答を入力してください:
            """
            ).lower()
            if DEBUG:
                display(Markdown(f"### デバッグ: 回答者の応答:"))
                display(Markdown(f"DEBUG: {response}"))
            
            yes_no = re.findall(r'\b(yes|no)\b', response)
            if yes_no:
                responses.append(yes_no[0])

        if DEBUG:
            display(Markdown(f"DEBUG: すべての回答者の応答 {responses}"))
        return max(set(responses), key=responses.count)

    except Exception as e:
        print(f"回答者でエラー: {e}", file=sys.stderr)
        return "yes"

# ================推測者エージェント================
# このエージェントはゲーム履歴を使用して推測を行います。
# 一部の場合、LLMが関連する単語ループにハマるため、これはあまり役に立たないことがあります。
# 異なるプロンプト戦略を試すことや、推測をスキップすることが良いでしょう。
# パース戦略を使用すると、ゲームの応答が有効で理解可能であることを保証します。

def guess(observation: Observation) -> str:
    game_history = observation.get_history_text(skip_guesses=False)
    
    think = generate_text(f"""
    あなたは現在、推測者としてキーワードの情報に基づいた推測を行います。

== ゲーム履歴 ==
{game_history}
==================

    提案された関連する1つのカテゴリと、可能な推測を提案してください。
    あなたの根拠を説明してください。
    """
    )
    
    parse = generate_text(f"""
    以下の応答を確認し、次のベストな推測を教えてください。
    あまり情報がない場合は、野生の具体的な推測をしてください。
    質問のように推測を尋ねず、単語またはフレーズの推測を直接提供してください。

    概要: "{think}"

    あなたの推測はこちら:
    """
    ).lower()
    if DEBUG:
        print(f"ゲーム履歴: {game_history}")
        display(Markdown(f"### デバッグ: 推測者の思考:"))
        display(Markdown(think))
        display(Markdown(f"### デバッグ: 推測者のパース:"))
        display(Markdown(parse))
    
    return parse

#==============LLMメッセージ生成===============
# 変更したいパラメータがいくつかあります。
# temperatureは生成されるテキストのランダム性を設定します。
# 値の低い方がより決定論的で、高い方がより創造的です。
# max_new_tokensは生成されるテキストの長さを設定します。

    
def generate_text(prompt:str) -> str:
    sys_prompt = system_prompt
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(messages, 
                                         tokenize=False, 
                                         add_generation_prompt=True)
    
    inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(inputs.input_ids,
                                   max_new_tokens=350, 
                                   do_sample=True, 
                                   temperature=0.1) #これは生成テキストのランダム性/創造性を設定します。低いほど決定論的です。
    
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

#================KAGGLE環境用エージェントの初期化================
# 評価中にモデルと対戦してエージェントの1つを'input'に変更すると楽しさが増します！
# ボーナスとして、微調整データセットを作成する可能性が開かれます！素晴らしいですね！
# 評価セッションを微調整データセットの作成セッションにすることを検討できます。これにより、出力をデータフレームに保存できます。
# 無限の可能性がありますが、時間に制約があります。  

agent = None

def observe(obs: t.Any) -> str:
    global agent
    observation = Observation(**obs.__dict__)

    try:
        match observation.turnType:
            case "ask":
                agent = ask(observation)
                return agent
            case "answer":
                agent = answer(observation)
                return agent
            case "guess":
                agent = guess(observation)
                return agent

            case _:
                raise ValueError("不明なターンタイプ")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    return observe(obs)

# =========================TORCHバックエンドを有効化===========================
# このノートブックでスクリプトを実行するときにエラーが発生することがあります。
# これらのバックエンド設定は、エラーなしでスクリプトが実行されることを保証します。やったね！

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Submission - Tarball
This installs and zips the submission into the format that kaggle expects. It zips the model, the script, and the submission folder. Note, if you add pip installs to the script, you will need to add them to the submission tarball. Install them to /tmp/submission/lib to add them to the submission tarball.

</div>
<div class="column-right">

# 日本語訳

# 提出 - タールボール
このインストールと圧縮は、kaggleが期待するフォーマットに提出物を作成します。モデル、スクリプト、および提出フォルダを含めて圧縮します。pipインストールをスクリプトに追加する場合は、それらを提出tarballにも追加する必要があります。提出tarballに追加するために/tmp/submission/libにインストールしてください。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!apt install pigz pv
```

</div>
<div class="column-right">

# 日本語訳

```python
!apt install pigz pv
```

</div>
</details>

In [ ]:
!apt install pigz pv

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!tar --use-compress-program='pigz --fast' \
     -cf submission.tar.gz \
     -C /kaggle/working main.py \
     -C /tmp/submission/ \
     .
```

</div>
<div class="column-right">

# 日本語訳

```python
!tar --use-compress-program='pigz --fast' \
     -cf submission.tar.gz \
     -C /kaggle/working main.py \
     -C /tmp/submission/ \
     .
```

</div>
</details>

In [ ]:
!tar --use-compress-program='pigz --fast' \
     -cf submission.tar.gz \
     -C /kaggle/working main.py \
     -C /tmp/submission/ \
     .

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Evaluation
If perform eval is set to True, the code below downloads the latest public keywords from the kaggle repository and generates a random keyword. This avoids initializing the kaggle environement which can cause out of memory errors. You will be asked if you'd like to enable verbose debug mode. This will print the prompts and responses from the LLM. This can be helpful when modifying the prompt structure, when calling objects within the prompt, or with multi-step prompt structures where the output is not final resposne in the game. 

* When running eval, make sure your have the accelerator turned on. 
* When you run an eval session, you may want to alter and test your prompt structure and try different strategies. If you do this, I recommend commenting out the top of the script that initializes the LLM (model and tokenizer). If you do not, you will load the model each time and you will run out of memory.
* As you experiment with different models, check the time it take to get a response. The competition limits reponse time to 60 seconds! (make sure you have a GPU accelaration turned on first).

### perform_eval? What's that for? 
If you set the variable 'perform_eval' to True, when you run all your cells or commit your notebook your agents will be evaluated in a simulated 20 questions game inside the notebook against itself with a keyword from the Kaggle repository. Yay!

If you set this to True, know that you will be prompted for input in about 2-3 minutes to see if you'd like to do debug mode or not.

Set to false to skip the evaluation session for quick commit and submission
#### **Make sure to turn on GPU acceleration for speedy evaluation.**

</div>
<div class="column-right">

# 日本語訳

# 評価
perform_evalがTrueに設定されている場合、以下のコードはKaggleリポジトリから最新の公的キーワードをダウンロードし、ランダムなキーワードを生成します。これにより、Kaggle環境の初期化を避けられ、メモリエラーを回避できます。デバッグモードを有効にするかどうかが求められます。これは、プロンプト構造を変更したり、プロンプト内のオブジェクトを呼び出したり、マルチステッププロンプト構造で最終応答がゲーム内にない場合に役立ちます。

* 評価を実行する際は、必ずアクセラレーターをオンにしてください。 
* 評価セッションを実行する場合、プロンプト構造を変えて異なる戦略を試したくなることがあります。その場合、LLM（モデルとトークナイザー）の初期化を行うスクリプトのトップをコメントアウトすることをお勧めします。そうしないと、毎回モデルを読み込んでしまい、メモリ不足になります。
* 異なるモデルを試しているときは、応答が得られるまでにかかる時間を確認してください。コンペティションは応答時間を60秒に制限しています！（最初にGPU加速が有効化されていることを確認してください）。

### perform_eval? これは何のためですか？
perform_evalをTrueに設定すると、すべてのセルを実行したりノートブックをコミットしたりすると、エージェントがKaggleリポジトリのキーワードを使用してノートブック内で自己対戦する20の質問ゲームで評価されます。やった！

これをTrueに設定すると、デバッグモードかどうかを聞かれるため、約2〜3分後にプロンプトが表示されます。

評価セッションをスキップしたい場合は、Falseに設定して迅速なコミットと提出を行ってください。
#### **評価の迅速化のためにGPU加速を必ずオンにしてください。**


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
perform_eval = False
```

</div>
<div class="column-right">

# 日本語訳

```python
perform_eval = False
```

</div>
</details>

In [ ]:
perform_eval = False

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
if perform_eval:
    import requests
    import json
    import re
    import typing as t
    import random
    import time
    from IPython.display import display, Markdown
    import signal

    # ===============FETCH LATEST KEYWORDS FROM KAGGLE GITHUB================
    # This does not train the model to look for the keywords.
    # This just fetches the list so it can be used in the evaluation.
    
    url = "https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py"
    response = requests.get(url)

    if response.status_code == 200:
        match = re.search(r'KEYWORDS_JSON = """(.*?)"""', response.text, re.DOTALL)
        if match:
            json_str = match.group(1)
            keywords_dict = json.loads(json_str)
        else:
            print("Could not find the KEYWORDS_JSON variable in the file.")
    else:
        print("Request failed with status code:", response.status_code)
        import time

    def select_random_keyword(keywords_dict: t.List[dict]) -> str:
        category = random.choice(keywords_dict)
        keyword_dict = random.choice(category['words'])
        return keyword_dict['keyword'].lower()
    
    #===============20 QUESTIONS EVALUATION SESSION=====================
    def input_timeout(prompt, timeout):
        def timeout_handler(signum, frame):
            raise TimeoutError("Input timed out after {} seconds".format(timeout))

        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)

        try:
            user_input = input(prompt)
            signal.alarm(0)
            return user_input
        except TimeoutError as e:
            print(e)
            return None

    timeout = 10

    try:
        DEBUG_input = input_timeout("Enable verbose debug mode? (y/n) [default is n] ", timeout)
        DEBUG = DEBUG_input.lower() == 'y' if DEBUG_input else False
    except:
        DEBUG = False
        print("No input received, defaulting to false.")

    class MockObservation:
        def __init__(self, step: int, role: str, turnType: str, keyword: str, category: str, questions: list[str], answers: list[str], guesses: list[str]):
            self.step = step
            self.role = role
            self.turnType = turnType
            self.keyword = keyword
            self.category = category
            self.questions = questions
            self.answers = answers
            self.guesses = guesses

    def test_20_questions():
        global DEBUG
        step = 0
        role = "answerer"
        turnType = "ask"
        keyword = select_random_keyword(keywords_dict)
        category = ""
        questions = []
        answers = []
        guesses = []
        display(Markdown("# Starting 20 questions eval game..."))
        display(Markdown(f"### **Keyword:** {keyword}"))

        for i in range(60):
            obs = MockObservation(step, role, turnType, keyword, category, questions, answers, guesses)

            start_time = time.time()
            response = agent_fn(obs, None)
            end_time = time.time()

            response_time = end_time - start_time
            if response_time > 60:
                display(Markdown(f"**WARNING:** Response time too long and may be disqualified from the game: {response_time:.2f} sec. Make sure you have GPU acceleration enabled in the session options on the right side panel."))
                break

            # Record the response in the appropriate list
            if turnType == 'ask':
                questions.append(response)
                turnType = 'answer'
            elif turnType == 'answer':
                answers.append(response)
                turnType = 'guess'
            elif turnType == 'guess':
                guesses.append(response)
                if response.lower() == keyword.lower():
                    display(Markdown(f"## **Keyword '{keyword}' guessed correctly! Ending game.**"))
                    break
                turnType = 'ask'
                step += 1

            display(Markdown(f"Step {step} | Response: {response} | {response_time:.2f} sec"))
        display(Markdown(f"Final Questions: {', '.join(questions)}"))
        display(Markdown(f"Final Answers: {', '.join(answers)}"))
        display(Markdown(f"Final Guesses: {', '.join(guesses)}"))

    # Run the test
    test_20_questions()
```

</div>
<div class="column-right">

# 日本語訳

```python
if perform_eval:
    import requests
    import json
    import re
    import typing as t
    import random
    import time
    from IPython.display import display, Markdown
    import signal

    # ===============KAGGLE GITHUBから最新のキーワードを取得================
    # これはモデルがキーワードを探すためにトレーニングされることはありません。
    # これはただ評価に使用されるリストを取得するだけです。
    
    url = "https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py"
    response = requests.get(url)

    if response.status_code == 200:
        match = re.search(r'KEYWORDS_JSON = """(.*?)"""', response.text, re.DOTALL)
        if match:
            json_str = match.group(1)
            keywords_dict = json.loads(json_str)
        else:
            print("ファイル内でKEYWORDS_JSON変数を見つけることができませんでした。")
    else:
        print("リクエストが失敗しました。ステータスコード:", response.status_code)
        import time

    def select_random_keyword(keywords_dict: t.List[dict]) -> str:
        category = random.choice(keywords_dict)
        keyword_dict = random.choice(category['words'])
        return keyword_dict['keyword'].lower()
    
    #===============20の質問評価セッション=====================
    def input_timeout(prompt, timeout):
        def timeout_handler(signum, frame):
            raise TimeoutError("入力タイムアウト: {}秒後".format(timeout))

        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)

        try:
            user_input = input(prompt)
            signal.alarm(0)
            return user_input
        except TimeoutError as e:
            print(e)
            return None

    timeout = 10

    try:
        DEBUG_input = input_timeout("詳細なデバッグモードを有効にしますか？ (y/n) [デフォルトはn] ", timeout)
        DEBUG = DEBUG_input.lower() == 'y' if DEBUG_input else False
    except:
        DEBUG = False
        print("入力が受け取られませんでした。デフォルトでfalseに設定します。")

    class MockObservation:
        def __init__(self, step: int, role: str, turnType: str, keyword: str, category: str, questions: list[str], answers: list[str], guesses: list[str]):
            self.step = step
            self.role = role
            self.turnType = turnType
            self.keyword = keyword
            self.category = category
            self.questions = questions
            self.answers = answers
            self.guesses = guesses

    def test_20_questions():
        global DEBUG
        step = 0
        role = "answerer"
        turnType = "ask"
        keyword = select_random_keyword(keywords_dict)
        category = ""
        questions = []
        answers = []
        guesses = []
        display(Markdown("# 20の質問評価ゲームを開始します..."))
        display(Markdown(f"### **キーワード:** {keyword}"))

        for i in range(60):
            obs = MockObservation(step, role, turnType, keyword, category, questions, answers, guesses)

            start_time = time.time()
            response = agent_fn(obs, None)
            end_time = time.time()

            response_time = end_time - start_time
            if response_time > 60:
                display(Markdown(f"**警告:** 応答時間が長すぎます。このゲームから失格になる可能性があります: {response_time:.2f}秒。右側のパネルでセッションオプションを用いてGPU加速が有効になっていることを確認してください。"))
                break

            # 適切なリストに応答を記録
            if turnType == 'ask':
                questions.append(response)
                turnType = 'answer'
            elif turnType == 'answer':
                answers.append(response)
                turnType = 'guess'
            elif turnType == 'guess':
                guesses.append(response)
                if response.lower() == keyword.lower():
                    display(Markdown(f"## **キーワード '{keyword}'が正しく推測されました！ ゲームを終了します。**"))
                    break
                turnType = 'ask'
                step += 1

            display(Markdown(f"ステップ {step} | 応答: {response} | {response_time:.2f}秒"))
        display(Markdown(f"最終質問: {', '.join(questions)}"))
        display(Markdown(f"最終回答: {', '.join(answers)}"))
        display(Markdown(f"最終推測: {', '.join(guesses)}"))

    # テストを実行
    test_20_questions()
```

</div>
</details>

In [ ]:
if perform_eval:
    import requests
    import json
    import re
    import typing as t
    import random
    import time
    from IPython.display import display, Markdown
    import signal

    # ===============KAGGLE GITHUBから最新のキーワードを取得================
    # これはモデルがキーワードを探すためにトレーニングされることはありません。
    # これはただ評価に使用されるリストを取得するだけです。
    
    url = "https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py"
    response = requests.get(url)

    if response.status_code == 200:
        match = re.search(r'KEYWORDS_JSON = """(.*?)"""', response.text, re.DOTALL)
        if match:
            json_str = match.group(1)
            keywords_dict = json.loads(json_str)
        else:
            print("ファイル内でKEYWORDS_JSON変数を見つけることができませんでした。")
    else:
        print("リクエストが失敗しました。ステータスコード:", response.status_code)
        import time

    def select_random_keyword(keywords_dict: t.List[dict]) -> str:
        category = random.choice(keywords_dict)
        keyword_dict = random.choice(category['words'])
        return keyword_dict['keyword'].lower()
    
    #===============20の質問評価セッション=====================
    def input_timeout(prompt, timeout):
        def timeout_handler(signum, frame):
            raise TimeoutError("入力タイムアウト: {}秒後".format(timeout))

        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)

        try:
            user_input = input(prompt)
            signal.alarm(0)
            return user_input
        except TimeoutError as e:
            print(e)
            return None

    timeout = 10

    try:
        DEBUG_input = input_timeout("詳細なデバッグモードを有効にしますか？ (y/n) [デフォルトはn] ", timeout)
        DEBUG = DEBUG_input.lower() == 'y' if DEBUG_input else False
    except:
        DEBUG = False
        print("入力が受け取られませんでした。デフォルトでfalseに設定します。")

    class MockObservation:
        def __init__(self, step: int, role: str, turnType: str, keyword: str, category: str, questions: list[str], answers: list[str], guesses: list[str]):
            self.step = step
            self.role = role
            self.turnType = turnType
            self.keyword = keyword
            self.category = category
            self.questions = questions
            self.answers = answers
            self.guesses = guesses

    def test_20_questions():
        global DEBUG
        step = 0
        role = "answerer"
        turnType = "ask"
        keyword = select_random_keyword(keywords_dict)
        category = ""
        questions = []
        answers = []
        guesses = []
        display(Markdown("# 20の質問評価ゲームを開始します..."))
        display(Markdown(f"### **キーワード:** {keyword}"))

        for i in range(60):
            obs = MockObservation(step, role, turnType, keyword, category, questions, answers, guesses)

            start_time = time.time()
            response = agent_fn(obs, None)
            end_time = time.time()

            response_time = end_time - start_time
            if response_time > 60:
                display(Markdown(f"**警告:** 応答時間が長すぎます。このゲームから失格になる可能性があります: {response_time:.2f}秒。右側のパネルでセッションオプションを用いてGPU加速が有効になっていることを確認してください。"))
                break

            # 適切なリストに応答を記録
            if turnType == 'ask':
                questions.append(response)
                turnType = 'answer'
            elif turnType == 'answer':
                answers.append(response)
                turnType = 'guess'
            elif turnType == 'guess':
                guesses.append(response)
                if response.lower() == keyword.lower():
                    display(Markdown(f"## **キーワード '{keyword}'が正しく推測されました！ ゲームを終了します。**"))
                    break
                turnType = 'ask'
                step += 1

            display(Markdown(f"ステップ {step} | 応答: {response} | {response_time:.2f}秒"))
        display(Markdown(f"最終質問: {', '.join(questions)}"))
        display(Markdown(f"最終回答: {', '.join(answers)}"))
        display(Markdown(f"最終推測: {', '.join(guesses)}"))

    # テストを実行
    test_20_questions()